In [1]:
import requests
r = requests.get('https://raw.githubusercontent.com/grbruns/cst383/master/alice.txt')
data_raw = r.content
r = open('input.txt', 'r')
data_raw = r.read()

#o = open('image_labels.txt', 'r')
#image_labels = eval(o.read())

In [2]:
import tensorflow as tf
import numpy as np
from datetime import *
import itertools
import nltk
from itertools import product
import re
import pandas as pd
import time

C:\Users\snoel\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
words_list = nltk.download('words')

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Kyle\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [6]:
from nltk.corpus import words
words_list = words.words()

In [7]:
from nltk.corpus import wordnet

In [8]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Kyle\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [9]:
wordnet.synsets("Umongly")

[]

In [3]:
#data = open('input.txt', 'r').read() # should be simple plain text file
data = str(data_raw)
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('data has %d characters, %d unique.' % (data_size, vocab_size))

#helper function to convert char to index
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }


data has 144393 characters, 69 unique.


In [4]:
print(data)


ALICE'S ADVENTURES IN WONDERLAND

Lewis Carroll

CHAPTER I. Down the Rabbit-Hole

Alice was beginning to get very tired of sitting by her sister on the
bank, and of having nothing to do: once or twice she had peeped into the
book her sister was reading, but it had no pictures or conversations in
it, 'and what is the use of a book,' thought Alice 'without pictures or
conversations?'

So she was considering in her own mind (as well as she could, for the
hot day made her feel very sleepy and stupid), whether the pleasure
of making a daisy-chain would be worth the trouble of getting up and
picking the daisies, when suddenly a White Rabbit with pink eyes ran
close by her.

There was nothing so VERY remarkable in that; nor did Alice think it so
VERY much out of the way to hear the Rabbit say to itself, 'Oh dear!
Oh dear! I shall be late!' (when she thought it over afterwards, it
occurred to her that she ought to have wondered at this, but at the time
it all seemed quite natural); but when t

In [5]:
print(len(data)/25)

5775.72


In [6]:
print(char_to_ix,'\n','\n', ix_to_char)

{'x': 0, 'l': 1, 'h': 2, 'M': 3, 'V': 4, 'r': 5, 'P': 6, 'U': 7, 'g': 8, 'e': 9, '.': 10, 'R': 11, 'a': 12, 'v': 13, 'd': 14, ':': 15, 'u': 16, '?': 17, 'H': 18, 'N': 19, 'S': 20, 'O': 21, 'f': 22, '_': 23, '-': 24, 'C': 25, 'W': 26, 'T': 27, ' ': 28, 'q': 29, '(': 30, '"': 31, 'B': 32, '[': 33, 'G': 34, 'n': 35, 'D': 36, ']': 37, 'K': 38, 'I': 39, ')': 40, 'Y': 41, 'w': 42, 'J': 43, "'": 44, 'o': 45, 'm': 46, 'p': 47, 'Z': 48, 'z': 49, 'Q': 50, 'y': 51, ';': 52, 'X': 53, 's': 54, 'b': 55, 'c': 56, 'F': 57, 'j': 58, 'k': 59, '*': 60, '!': 61, '\n': 62, 'L': 63, 't': 64, 'i': 65, ',': 66, 'A': 67, 'E': 68} 
 
 {0: 'x', 1: 'l', 2: 'h', 3: 'M', 4: 'V', 5: 'r', 6: 'P', 7: 'U', 8: 'g', 9: 'e', 10: '.', 11: 'R', 12: 'a', 13: 'v', 14: 'd', 15: ':', 16: 'u', 17: '?', 18: 'H', 19: 'N', 20: 'S', 21: 'O', 22: 'f', 23: '_', 24: '-', 25: 'C', 26: 'W', 27: 'T', 28: ' ', 29: 'q', 30: '(', 31: '"', 32: 'B', 33: '[', 34: 'G', 35: 'n', 36: 'D', 37: ']', 38: 'K', 39: 'I', 40: ')', 41: 'Y', 42: 'w', 43: '

# my_encoder
creates a dictionary of encoded arrays for each index in our vocabulary

In [7]:
my_encoder = {}
for t in range(vocab_size):
  my_encoder[t] = np.zeros((vocab_size, 1))
  my_encoder[t][t] = 1
#print(my_encoder.keys())
#print(xs)
print('encoded number shape:', my_encoder[3].ravel().shape)
print('integer 3 encoded: \n',my_encoder[3].ravel())

encoded number shape: (69,)
integer 3 encoded: 
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


# Fetch Batch
Below we have the fetch batch function.

this function takes batch_size and n_batches as input.

It will return a X_batch and y_batch in the shape of (# of batches, # of steps, # of inputs at step)
The # of inputs at step should be 69 (e.g. the value is encoded )

(10, 25, 69)

# Building Graph
first we start off with building a simple graph to test input.

We give it 25 stpes each with 1 input at each step encoded to a len(vocab_size)


In [8]:
class model_selector():
    '''
    model_selection using api calls
    '''
    
    
    def sample(self, len_sentence):
        indicies=[]
        ix = np.random.randint(vocab_size)
        indicies.append(ix)
        test_batch = np.zeros((1, len_sentence, self.vocab_size))
        
        for i in range(len_sentence):
            test_batch[0, i, :][ix] = 1
            p = self.prediction.eval(feed_dict = {self.X:test_batch[:, :i+1, :]})
            p = p[0][i].ravel()
            ix = np.random.choice(range(self.vocab_size), p=p)
            indicies.append(ix)
        return indicies
    
    def sample_last_model(self, len_sentence):
        #NEED TO FIGURE OUT HOW TO RESTORE AND GET PREDICTED OUTPUT
        saver = tf.train.Saver()
        with tf.Session() as sess:   
            saver.restore(sess,'my_model_final.ckpt')
            indicies=[]
            ix = np.random.randint(self.vocab_size)
            indicies.append(ix)
            test_batch = np.zeros((1, len_sentence, self.vocab_size))
            #print(test_batch.shape)
            for i in range(len_sentence):
                #print('i', i)
                #print('test_batch', test_batch.shape)
                test_batch[0, i, :][ix] = 1
                p = sess.run([self.prediction], feed_dict = {self.X:test_batch[:, :i+1, :]})
                #p = self.prediction.eval(feed_dict = {self.X:test_batch[:, i:i+1, :]})
                #print(p.shape)
                #print(p[0])
                p = p[0][0][i].ravel()
                ix = np.random.choice(range(self.vocab_size), p=p)
                indicies.append(ix)
            print('OUTPUT:',''.join([ix_to_char[ix] for ix in indicies]),'\n')
            print('---------------------------------------------------------------------------')

    def fetch_batch(self, n_batches, batch_size,):
        X_batch = []
        y_batch = []
        for _ in range(n_batches):
            p = np.random.randint(len(data)-batch_size, size=1)[0] 
            X_batch.append([self.my_encoder[char_to_ix[ch]].ravel() for ch in data[p:p+batch_size]])
            y_batch.append([char_to_ix[ch] for ch in data[p+1:p+batch_size+1]])
        return np.array(X_batch), np.array(y_batch)
    
    def initialize(self, parameters, grid_run):
        self.parameters=parameters
        self.args(layer_type=parameters['layer_type'], n_layers=parameters['n_layers'], 
                  vocab_size=parameters['vocab_size'], learning_rate=parameters['learning_rate'], 
                  n_steps=parameters['n_steps'], n_batches=parameters['n_batches'],
                  n_epochs=parameters['n_epochs'], verbose=parameters['verbose'],
                  len_sentence=parameters['len_sentence'], n_neurons=parameters['n_neurons'], grid_run = grid_run)

        print('---------------------------------------------------------------------------')
        print('MODEL INITIALIZING')
        X_batch, y_batch = self.fetch_batch(self.n_batches, self.batch_size)
        print('Batch Shape X:', X_batch.shape)
        print('Batch Shape y:', y_batch.shape)
        self.graph_build()
        print('MODEL INITIALIZED')
        print('---------------------------------------------------------------------------')
        self.train_model()
            
    def args(self, layer_type, n_layers, vocab_size, learning_rate, 
             n_steps, n_batches, n_epochs, verbose, len_sentence, n_neurons, grid_run):
        #GRAPH ARGS
        self.layer_type = layer_type
        self.n_layers = n_layers
        self.n_neurons = n_neurons
        self.n_inputs = vocab_size
        self.n_outputs = vocab_size 
        self.vocab_size = vocab_size
        self.learning_rate = learning_rate
        
        
        #Training Args
        self.n_steps = n_steps
        self.n_batches = n_batches
        self.batch_size = n_steps
        self.n_epochs = n_epochs
        self.len_sentence = len_sentence
        self.verbosity = verbose # give the number of steps to evaluate at
        
        #Configure local dependencies
        self.root_logdir = 'C:/Users/Kyle/tf_logs/'
        self.config = tf.ConfigProto()
        self.config.gpu_options.allow_growth = True
        
        self.grid_run = grid_run
        
        self.my_encoder = {}
        for t in range(vocab_size):
            self.my_encoder[t] = np.zeros((vocab_size, 1))
            self.my_encoder[t][t] = 1
    def retrieve_args(self):
        return [self.grid_run, self.parameters, self.loss_train, self.accuracy_train, self.loss_valid, self.accuracy_valid, self.output_arg]
    


    def graph_build(self):
        he_init = tf.keras.initializers.glorot_uniform(seed=None)
        tf.reset_default_graph()
        
        with tf.name_scope('inputs'):
            self.X = tf.placeholder(tf.float32, [None, None, self.n_inputs])
            self.y = tf.placeholder(tf.int64, [None, None])

        with tf.name_scope('cells'):
            if self.layer_type == 'LSTM':
                cells = [tf.nn.rnn_cell.LSTMCell(num_units=self.n_neurons, activation=tf.nn.leaky_relu) for _ in range(self.n_layers)]
            elif self.layer_type == 'GRU':
                cells = [tf.nn.rnn_cell.GRUCell(num_units=self.n_neurons, activation=tf.nn.leaky_relu) for _ in range(self.n_layers)]
            else:
                cells = [tf.nn.rnn_cell.BasicRNNCell(num_units=self.n_neurons, activation=tf.nn.leaky_relu) for _ in range(self.n_layers)]
            multi_cell = tf.contrib.rnn.MultiRNNCell(cells)
            outputs, states = tf.nn.dynamic_rnn(multi_cell, self.X, dtype=tf.float32)
            
        with tf.name_scope('outputs'):
            logits = tf.layers.dense(outputs, self.vocab_size,
                                     name='dense', activation = tf.nn.leaky_relu, 
                                     kernel_initializer = he_init)
            #logits2 = tf.layers.dense(outputs, self.vocab_size*3,
            #                         name='dense2', activation = tf.nn.leaky_relu, 
            #                         kernel_initializer = he_init)
            #dropout = tf.layers.dropout(logits2*2, rate = 0.2)
            #logits3 = tf.layers.dense(dropout, self.vocab_size,
            #                         name='dense3', activation = tf.nn.leaky_relu, 
            #                         kernel_initializer = he_init)
            #dropout2 = tf.layers.dropout(logits3, rate = 0.2)
            #logits4 = tf.layers.dense(dropout2, self.vocab_size,
            #                         name='dense4', activation = tf.nn.leaky_relu, 
            #                         kernel_initializer = he_init)

        with tf.name_scope('accuracy/evaluation'):
            self.output = tf.argmax(logits, axis=2, name='output')
            self.prediction = tf.nn.softmax(logits)
            self.correct_pred = tf.equal(self.output[self.n_batches-1], self.y[self.n_batches-1])
            self.accuracy = tf.reduce_mean(tf.cast(self.correct_pred, tf.float32))

        with tf.name_scope('training'):
            xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=self.y, name='loss')
            self.loss = tf.reduce_mean(xentropy)
            optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate)
            self.train_op = optimizer.minimize(self.loss)
            
        print('Graph Shape')
        print('X:', self.X.shape)
        print('y:', self.y.shape)
        print('outputs:', outputs.shape)
        print('states:', states[-1][1].shape)
        print('logits:', logits.shape)
        print('prediction', self.prediction.shape)
        print('output:', self.output.shape)
        
    def train_model(self):
        #setup the logging
        #now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
        #logdir = "{}/run-{}/".format(self.root_logdir, now)
        #loss_summary = tf.summary.scalar('loss', self.loss)
        #accuracy_summary = tf.summary.scalar('accuracy_train', self.accuracy)
        #loss_summary_valid = tf.summary.scalar('loss_valid', self.loss)
        #accuracy_summary_valid = tf.summary.scalar('accuracy_train_valid', self.accuracy)
        #file_write = tf.summary.FileWriter(logdir, tf.get_default_graph())
        
        saver = tf.train.Saver()
        init = tf.global_variables_initializer()

        with tf.Session(config=self.config) as sess:
            init.run()
            self.output_arg = []
            for _ in range(self.n_epochs):
                X_batch, y_batch = self.fetch_batch(self.n_batches, self.batch_size)
                X_valid, y_valid = self.fetch_batch(self.n_batches, self.batch_size)
                sess.run(self.train_op, feed_dict = {self.X: X_batch, self.y:y_batch})
                
                if _%375 == 0:
                    indicies = self.sample(self.len_sentence)
                    string = ''.join([ix_to_char[ix] for ix in indicies])
                    self.output_arg.append(string)
                    file_path_2 = "./my_model_final_machine_1_" + str(self.grid_run) + ".ckpt"
                    save_path = saver.save(sess, file_path_2)

                if _ %10 == 0:
                    #self.summary_str = loss_summary.eval(feed_dict={self.X: X_batch, self.y: y_batch})
                    #self.summary_str2 = accuracy_summary.eval(feed_dict={self.X: X_batch, self.y: y_batch})
                    #self.summary_str3 = loss_summary_valid.eval(feed_dict={self.X: X_valid, self.y: y_valid})
                    #self.summary_str4 = accuracy_summary_valid.eval(feed_dict={self.X: X_valid, self.y: y_valid})
                    #step = _ *self.n_batches
                    #file_write.add_summary(self.summary_str, step)   
                    #file_write.add_summary(self.summary_str2, step)
                    #file_write.add_summary(self.summary_str3, step)
                    #file_write.add_summary(self.summary_str4, step)
                    
                    self.accuracy_train = self.accuracy.eval(feed_dict = {self.X: X_batch, self.y:y_batch})
                    self.loss_train = self.loss.eval(feed_dict = {self.X: X_batch, self.y:y_batch})
                    self.accuracy_valid = self.accuracy.eval(feed_dict = {self.X: X_valid, self.y:y_valid})
                    self.loss_valid = self.loss.eval(feed_dict = {self.X: X_valid, self.y:y_valid})

                    
                    #if _ %self.verbosity == 0:
                    #    print('epoch', _)
                    #    print('accuracy', self.accuracy.eval(feed_dict = {self.X: X_batch, self.y:y_batch}))
                    #    print('loss', self.loss.eval(feed_dict = {self.X: X_batch, self.y:y_batch}))
                    #    print('y_batch examination:')
                    #    print('expected output:', ''.join([ix_to_char[ix] for ix in y_batch[self.n_batches-1].ravel()]))
                    #    print('  actual output:', ''.join([ix_to_char[ix] for ix in self.output.eval(
                    #        feed_dict = {self.X: X_batch, self.y:y_batch})[self.n_batches-1].ravel()]))

                    #    indicies = self.sample(self.len_sentence)
                    #    print()
                    #    print('OUTPUT:',''.join([ix_to_char[ix] for ix in indicies]),'\n')
                    #    #PERFORM EARLY STOPPING HERE
                    #    save_path = saver.save(sess, "./my_model_final.ckpt")

                    #    print('---------------------------------------------------------------------------')
    
        


In [200]:
if _ %self.verbosity == 0:
                        print('epoch', _)
                        print('accuracy', self.accuracy.eval(feed_dict = {self.X: X_batch, self.y:y_batch}))
                        print('loss', self.loss.eval(feed_dict = {self.X: X_batch, self.y:y_batch}))
                        print('y_batch examination:')
                        print('expected output:', ''.join([ix_to_char[ix] for ix in y_batch[self.n_batches-1].ravel()]))
                        print('  actual output:', ''.join([ix_to_char[ix] for ix in self.output.eval(
                            feed_dict = {self.X: X_batch, self.y:y_batch})[self.n_batches-1].ravel()]))

                        indicies = self.sample(self.len_sentence)
                        print()
                        print('OUTPUT:',''.join([ix_to_char[ix] for ix in indicies]),'\n')
                        #PERFORM EARLY STOPPING HERE
                        save_path = saver.save(sess, "./my_model_final.ckpt")

                        print('---------------------------------------------------------------------------')

NameError: name 'self' is not defined

In [212]:
#KARPATHY MODEL
selector = model_selector()
parameters = {
                #TRAINING ARGS
                'vocab_size': vocab_size,
                'n_steps': 25,
                'n_batches': 1,
                'n_epochs': 1800,
    
                #GRAPH ARGS
                #MUST CHANGE LAYER_TYPE MANUALLY
                'layer_type':'RNN',
                'n_layers': 1,
                'n_neurons': 100,
                'learning_rate': 0.0105,
    
                #ANALYSIS ARGS
                'verbose':50,
                'len_sentence':100      
            }

selector.initialize(parameters, 1)

---------------------------------------------------------------------------
MODEL INITIALIZING
Batch Shape X: (1, 25, 69)
Batch Shape y: (1, 25)
Graph Shape
X: (?, ?, 69)
y: (?, ?)
outputs: (?, ?, 100)
states: (100,)
logits: (?, ?, 69)
prediction (?, ?, 69)
output: (?, ?)
MODEL INITIALIZED
---------------------------------------------------------------------------


KeyboardInterrupt: 

In [214]:
print(selector.retrieve_args()[6])


['Mx.XZdX.U;Dazs_z-j)XR[MSL]gEhf!MvzrXWlaYt!Um)! cl(qbt,Z-LPnGx(U_vvns-(GfPI-\'zsi\nyis":ZaUTqjITg;iwS\'YE']


In [45]:
#TESTING NOW
selector = model_selector()
parameters = {
                #TRAINING ARGS
                'vocab_size': vocab_size,
                'n_steps': 25,
                'n_batches': 1,
                'n_epochs': 1600,
    
                #GRAPH ARGS
                #MUST CHANGE LAYER_TYPE MANUALLY
                'layer_type':'LSTM',
                'n_layers': 25,
                'n_neurons': 100,
                'learning_rate': 0.0005,
    
                #ANALYSIS ARGS
                'verbose':50,
                'len_sentence':100      
            }

selector.initialize(parameters, 1)

---------------------------------------------------------------------------
MODEL INITIALIZING
Batch Shape X: (1, 25, 69)
Batch Shape y: (1, 25)
Graph Shape
X: (?, ?, 69)
y: (?, ?)
outputs: (?, ?, 100)
states: (?, 100)
logits: (?, ?, 69)
prediction (?, ?, 69)
output: (?, ?)
MODEL INITIALIZED
---------------------------------------------------------------------------


KeyboardInterrupt: 

In [53]:
for _ in range(0,10):
    selector.sample_last_model(100)

NameError: name 'selector' is not defined

In [13]:
grid_search_parameters = {
                #TRAINING ARGS
                'vocab_size': [vocab_size],
                'n_steps': [step_count for step_count in range(25,100,25)],
                'n_batches': [n_batches for n_batches in range(1,61,10)],
                'n_epochs': [1500],
    
                #GRAPH ARGS
                'layer_type':['LSTM','GRU'],
                'n_layers': [n_layers for n_layers in range(1,200,10)],
                'n_neurons': [n_neurons for n_neurons in range(100,800,50)],
                'learning_rate': [0.1, 0.05, 0.01, 0.005, 0.001, 0.0005, 0.0001],
    
                #ANALYSIS ARGS
                'verbose':[50],
                'len_sentence':[100]      
            }

In [14]:
grid_search_parameters

{'vocab_size': [69],
 'n_steps': [25, 50, 75],
 'n_batches': [1, 11, 21, 31, 41, 51],
 'n_epochs': [1500],
 'layer_type': ['LSTM', 'GRU'],
 'n_layers': [1,
  11,
  21,
  31,
  41,
  51,
  61,
  71,
  81,
  91,
  101,
  111,
  121,
  131,
  141,
  151,
  161,
  171,
  181,
  191],
 'n_neurons': [100,
  150,
  200,
  250,
  300,
  350,
  400,
  450,
  500,
  550,
  600,
  650,
  700,
  750],
 'learning_rate': [0.1, 0.05, 0.01, 0.005, 0.001, 0.0005, 0.0001],
 'verbose': [50],
 'len_sentence': [100]}

In [15]:
keys, values = zip(*grid_search_parameters.items())
experiments = [dict(zip(keys, values)) for values in itertools.product(*values)]


In [16]:
print(len(experiments))
print(experiments[8000])

70560
{'vocab_size': 69, 'n_steps': 25, 'n_batches': 21, 'n_epochs': 1500, 'layer_type': 'LSTM', 'n_layers': 11, 'n_neurons': 500, 'learning_rate': 0.0001, 'verbose': 50, 'len_sentence': 100}


In [184]:
#d = pd.DataFrame(data = {'parameters':[],'loss':[],'accuracy':[],'loss_valid':[],'accuracy_valid':[], 'sentences':[]})
d = pd.DataFrame(columns = ['parameters','loss','accuracy','loss_valid','accuracy_valid', 'sentences'])
d.loc[0] = selector.retrieve_args()
d

,parameters,loss,accuracy,loss_valid,accuracy_valid,sentences
0,"{'vocab_size': 69, 'n_steps': 25, 'n_batches':...",3.229609,0.24,2.419544,0.32,"[ g""Uo""OkaDFhuCAVO[u(VHyczX!d""f:AnA(eo?,H""""hHC..."


In [ ]:
#PERFORM GRID SEARCH
#24 hour loop
dataframe = pd.DataFrame(columns = ['run','parameters','loss','accuracy','loss_valid','accuracy_valid', 'sentences'])
selector = model_selector()
timeout = time.time() + 60*720 # 720 minutes from now # 12 hours
while True:
    for i in range(0,len(experiments)):
        ix = np.random.randint(len(experiments))
        
        selector.initialize(experiments[ix], i)
        args = selector.retrieve_args()
        dataframe.loc[i] = args
        
        file_path = 'dataframe_machine_1_' + str(i) + '.csv'
        dataframe.to_csv(file_path)
            
        if time.time() > timeout:
            file_path = 'dataframe_machine_1_' + str(i) + '.csv'
            dataframe.to_csv(file_path)
            break


---------------------------------------------------------------------------
MODEL INITIALIZING
Batch Shape X: (21, 75, 69)
Batch Shape y: (21, 75)
Graph Shape
X: (?, ?, 69)
y: (?, ?)
outputs: (?, ?, 700)
states: (700,)
logits: (?, ?, 69)
prediction (?, ?, 69)
output: (?, ?)
MODEL INITIALIZED
---------------------------------------------------------------------------


# JUNK CODE NEEDED FOR LATER TRIALS

In [95]:
#CLOSE TO KARPATHY
with tf.device('/device:GPU:1'):
    #TRAINING SESSSION ARGS
    n_steps = 25
    n_batches = 13
    batch_size = n_steps
    n_epochs = 10000000000000000000000000000000000000000000000000
    len_sentence = 100


    #GRAPH ARGS

    n_inputs = vocab_size
    n_neurons = 100
    n_outputs = vocab_size 
    learning_rate = 0.0008
    he_init = tf.keras.initializers.glorot_uniform(seed=None)

    tf.reset_default_graph()

    X = tf.placeholder(tf.float32, [None, None, n_inputs])
    y = tf.placeholder(tf.int64, [None, None])

    cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, activation=tf.nn.leaky_relu)
    outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

    logits = tf.layers.dense(outputs, vocab_size, name='dense', activation = tf.nn.leaky_relu, kernel_initializer = he_init)

    #accuracy/evaluation
    output = tf.argmax(logits, axis=2, name='output') # Returns the index with the largest value across axes of a tensor. (deprecated arguments)
    prediction = tf.nn.softmax(logits)
    correct_pred = tf.equal(output[n_batches-1], y[n_batches-1])
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))



    #training
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y, name='loss')
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
    train_op = optimizer.minimize(loss)

    print('X:', X.shape)
    print('y:', y.shape)
    print('outputs:', outputs.shape)
    print('states:', states.shape)
    print('logits:', logits.shape)
    print('prediction', prediction.shape)
    print('output:', output.shape)


X: (?, ?, 69)
y: (?, ?)
outputs: (?, ?, 100)
states: (?, 100)
logits: (?, ?, 69)
prediction (?, ?, 69)
output: (?, ?)


In [94]:
####RUNNING TEST'S ON NEW CODE FORMAT#########
tf.reset_default_graph()
n_steps = 25
n_inputs = vocab_size
n_neurons = 100
n_outputs = vocab_size 



X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_steps, n_outputs])
#y = tf.placeholder(tf.float32, [None, n_steps, n_outputs])

cell = tf.contrib.rnn.OutputProjectionWrapper(tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, activation=tf.nn.relu), output_size= n_outputs)
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
#print(states.shape)



print('outputs:', outputs.shape)
print('states:', states.shape)
loss = tf.reduce_mean(tf.square(outputs-y))
print('loss:', loss.shape)

optimizer = tf.train.AdamOptimizer()
training_op = optimizer.minimize(loss)
correct_pred = tf.equal(outputs, y)
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))




outputs: (?, 25, 69)
states: (?, 100)
loss: ()


In [0]:
learning_rate = 0.001
loss = tf.reduce_mean(tf.square(outputs - y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()

In [0]:
n_iterations = 1500
batch_size = 25
with tf.Session() as sess:
     init.run()
    for iteration in range(n_iterations):
        X_batch, y_batch = [...] # fetch the next training batch
        sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        if iteration % 100 == 0:
            mse = loss.eval(feed_dict={X: X_batch, y: y_batch})
            print(iteration, "\tMSE:", mse)

In [0]:
tf.reset_default_graph()

n_steps = 28
n_inputs = 28
n_neurons = 150
n_outputs = 10

learning_rate = 0.001

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.int32, [None])

basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)

logits = tf.layers.dense(states, n_outputs)
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                          logits=logits)
loss = tf.reduce_mean(xentropy)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.


In [0]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")
X_test = mnist.test.images.reshape((-1, n_steps, n_inputs))
y_test = mnist.test.labels

In [0]:
n_epochs = 100
batch_size = 150

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            X_batch = X_batch.reshape((-1, n_steps, n_inputs))
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)

In [0]:
X_batch, y_batch = mnist.train.next_batch(150)
print(X_batch.shape)
print(y_batch.shape)
print(X_batch.reshape((-1, n_steps, n_inputs)).shape)


(150, 784)
(150,)
(150, 28, 28)


(150, 784)

In [0]:
"""
Minimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)
BSD License
"""
import numpy as np

# data I/O
data = open('input.txt', 'r').read() # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print ('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

def lossFun(inputs, targets, hprev):
  """
  inputs,targets are both list of integers.
  hprev is Hx1 array of initial hidden state
  returns the loss, gradients on model parameters, and last hidden state
  """
  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = np.copy(hprev)
  loss = 0
  # forward pass
  for t in range(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
    xs[t][inputs[t]] = 1
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
  #print('xs', xs[t].shape)
  print(loss)
  # backward pass: compute gradients going backwards
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(range(len(inputs))):
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
    dWhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(Why.T, dy) + dhnext # backprop into h
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)
    dWhh += np.dot(dhraw, hs[t-1].T)
    dhnext = np.dot(Whh.T, dhraw)
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

def sample(h, seed_ix, n):
  """ 
  sample a sequence of integers from the model 
  h is memory state, seed_ix is seed letter for first time step
  """
  x = np.zeros((vocab_size, 1))
  x[seed_ix] = 1
  ixes = []
  for t in range(n):
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    p = np.exp(y) / np.sum(np.exp(y))
    #ix is assigned to a random vocab_word according to the probabilite associated with it sotred in p.ravel()
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    
    x = np.zeros((vocab_size, 1))
    x[ix] = 1
    ixes.append(ix)
    print('p probabilites of each output of encoded array: ', p.shape, '\n,', p) 
    print('ix a randomly chosent index from the encoded array:', ix)
    print('x: decoded array where ix is set to 1', x.shape, '\n', x)
  print(ixes)
  return ixes

n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
while True:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  if p+seq_length+1 >= len(data) or n == 0: 
    hprev = np.zeros((hidden_size,1)) # reset RNN memory
    p = 0 # go from start of data
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]
  #print(inputs)

  # sample from the model now and then
  if n % 100 == 0:
    sample_ix = sample(hprev, inputs[0], 200)
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    print ('----\n %s \n----' % (txt, ))

  # forward seq_length characters through the net and fetch gradient
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  if n % 100 == 0: print ('iter %d, loss: %f' % (n, smooth_loss)) # print progress
  
  # perform parameter update with Adagrad
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

  p += seq_length # move data pointer
  n += 1 # iteration counter 

data has 144393 characters, 69 unique.
p (69, 1) 
, [[0.01447189]
 [0.01447933]
 [0.01447747]
 [0.01450613]
 [0.01450563]
 [0.01449442]
 [0.01448878]
 [0.01449321]
 [0.01449244]
 [0.01448215]
 [0.01449094]
 [0.0144675 ]
 [0.01451181]
 [0.01451356]
 [0.01448788]
 [0.01448305]
 [0.01449686]
 [0.01450552]
 [0.01450441]
 [0.01451002]
 [0.01445876]
 [0.0144961 ]
 [0.01447215]
 [0.01447614]
 [0.0145021 ]
 [0.01450716]
 [0.01447615]
 [0.01450592]
 [0.01450014]
 [0.01450741]
 [0.0145084 ]
 [0.01446877]
 [0.01451504]
 [0.01448796]
 [0.01447253]
 [0.01448199]
 [0.01448799]
 [0.01449064]
 [0.0144917 ]
 [0.0145306 ]
 [0.01448446]
 [0.01448339]
 [0.01449702]
 [0.01449384]
 [0.01449525]
 [0.01450165]
 [0.01449296]
 [0.01447371]
 [0.01447765]
 [0.01448719]
 [0.01446638]
 [0.01448741]
 [0.01446502]
 [0.01449184]
 [0.01449419]
 [0.01451569]
 [0.01448261]
 [0.01451716]
 [0.01450301]
 [0.01448823]
 [0.01448576]
 [0.01448503]
 [0.01449788]
 [0.01449861]
 [0.01451514]
 [0.01452645]
 [0.01448764]
 [0.014495

x (69, 1) 
 [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]]
p (69, 1) 
, [[0.01450516]
 [0.01447298]
 [0.01447078]
 [0.01449613]
 [0.01448799]
 [0.01447747]
 [0.01448337]
 [0.01449852]
 [0.01449403]
 [0.01449091]
 [0.0145072 ]
 [0.01447622]
 [0.01448475]
 [0.01449715]
 [0.01450535]
 [0.01445959]
 [0.01450182]
 [0.01449749]
 [0.01451859]
 [0.01448691]
 [0.0144978 ]
 [0.01447757]
 [0.0144898 ]
 [0.01450261]
 [0.01452161]
 [0.01451741]
 [0.01448027]
 [0.01448986]
 [0.01447849]
 [0.01447403]
 [0.01449192]
 [0.01448509]
 [0.01452421]
 [0.01449477]
 [0.01449345]
 [0.01450296]
 [0.01449526]
 [0.01450716]
 [0.01448412]
 [0.01448904]


p (69, 1) 
, [[0.01451223]
 [0.01449241]
 [0.01449808]
 [0.01447325]
 [0.01446539]
 [0.01447509]
 [0.01450606]
 [0.01451366]
 [0.0144953 ]
 [0.01448771]
 [0.01448687]
 [0.01452257]
 [0.0144775 ]
 [0.01449399]
 [0.01448911]
 [0.01447086]
 [0.0145224 ]
 [0.01445856]
 [0.01452322]
 [0.01448778]
 [0.01446309]
 [0.01449753]
 [0.01452369]
 [0.01450423]
 [0.01452811]
 [0.01450922]
 [0.01449407]
 [0.01450769]
 [0.01451538]
 [0.01449575]
 [0.01445464]
 [0.01449393]
 [0.01450316]
 [0.01449284]
 [0.01447612]
 [0.01449264]
 [0.0144892 ]
 [0.01452359]
 [0.01450642]
 [0.01452276]
 [0.01446327]
 [0.01450932]
 [0.01448666]
 [0.01448665]
 [0.01448053]
 [0.01448237]
 [0.0145171 ]
 [0.01448831]
 [0.01447407]
 [0.01448531]
 [0.01450039]
 [0.01449345]
 [0.01448549]
 [0.01450583]
 [0.01450353]
 [0.01448341]
 [0.01448622]
 [0.01448546]
 [0.01450629]
 [0.01448379]
 [0.01449967]
 [0.01448255]
 [0.0144947 ]
 [0.01448959]
 [0.01446679]
 [0.01448224]
 [0.01449063]
 [0.01447853]
 [0.01446172]]
x (69, 1) 
 [[0.]
 [

 [0.]]
p (69, 1) 
, [[0.01448274]
 [0.01450664]
 [0.014495  ]
 [0.01447069]
 [0.01450495]
 [0.01449621]
 [0.01448747]
 [0.0144992 ]
 [0.01448767]
 [0.01448506]
 [0.01451721]
 [0.01450973]
 [0.01449597]
 [0.01447895]
 [0.01449458]
 [0.01451096]
 [0.01449157]
 [0.01448558]
 [0.01450786]
 [0.01447818]
 [0.01448604]
 [0.01451151]
 [0.01447653]
 [0.01448705]
 [0.01450217]
 [0.01447514]
 [0.01450047]
 [0.01448214]
 [0.01447382]
 [0.01448115]
 [0.01449112]
 [0.01448677]
 [0.0144462 ]
 [0.01448844]
 [0.01448587]
 [0.01449738]
 [0.01449829]
 [0.01450314]
 [0.01447727]
 [0.01449687]
 [0.01449048]
 [0.0144689 ]
 [0.01450167]
 [0.01448709]
 [0.01450366]
 [0.01451355]
 [0.01448622]
 [0.01447609]
 [0.01447911]
 [0.01449858]
 [0.01448897]
 [0.01449759]
 [0.01449734]
 [0.01450719]
 [0.01449771]
 [0.01450256]
 [0.01451309]
 [0.01449099]
 [0.01449299]
 [0.01452342]
 [0.01447515]
 [0.01447507]
 [0.01447677]
 [0.01449033]
 [0.01450385]
 [0.01450808]
 [0.01451275]
 [0.01448755]
 [0.01451966]]
x (69, 1) 
 [

x (69, 1) 
 [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
p (69, 1) 
, [[0.0145074 ]
 [0.01447838]
 [0.01449439]
 [0.01447768]
 [0.0145046 ]
 [0.01452297]
 [0.01451485]
 [0.01449766]
 [0.014491  ]
 [0.01449487]
 [0.01448307]
 [0.01450049]
 [0.01449189]
 [0.01449184]
 [0.01449357]
 [0.01448489]
 [0.01447952]
 [0.01447541]
 [0.01446003]
 [0.01449644]
 [0.01450984]
 [0.01447837]
 [0.0144978 ]
 [0.01448522]
 [0.01450453]
 [0.01448575]
 [0.01449824]
 [0.01451904]
 [0.0144999 ]
 [0.01450283]
 [0.01449366]
 [0.01448426]
 [0.01449102]
 [0.01446753]
 [0.01452107]
 [0.01446952]
 [0.01450879]
 [0.01449569]
 [0.01449704]
 [0.01447092]


p (69, 1) 
, [[0.01448227]
 [0.01450825]
 [0.01452615]
 [0.01450362]
 [0.01449647]
 [0.01447878]
 [0.01451706]
 [0.01452009]
 [0.01448515]
 [0.01449753]
 [0.01449928]
 [0.01449419]
 [0.01449627]
 [0.01445637]
 [0.0144855 ]
 [0.01448985]
 [0.01450875]
 [0.01446268]
 [0.01450051]
 [0.01449547]
 [0.01448362]
 [0.01448057]
 [0.01448454]
 [0.01450889]
 [0.01450876]
 [0.01448572]
 [0.01449807]
 [0.01449721]
 [0.01447546]
 [0.01447024]
 [0.01449617]
 [0.01447483]
 [0.01451071]
 [0.01451271]
 [0.0145157 ]
 [0.01450622]
 [0.01449411]
 [0.0144995 ]
 [0.01447932]
 [0.01453111]
 [0.01447841]
 [0.01446772]
 [0.01448553]
 [0.01447624]
 [0.01446834]
 [0.0144838 ]
 [0.01446958]
 [0.01447774]
 [0.01449644]
 [0.01451283]
 [0.01450016]
 [0.01451146]
 [0.01448034]
 [0.01449751]
 [0.0144953 ]
 [0.01451213]
 [0.01450642]
 [0.01446518]
 [0.01446312]
 [0.01447986]
 [0.01447391]
 [0.01449355]
 [0.01447713]
 [0.01448072]
 [0.01451524]
 [0.01451568]
 [0.01448023]
 [0.01449798]
 [0.01451977]]
x (69, 1) 
 [[0.]
 [

iter 0, loss: 105.852672
104.34100228666314
225.3787161453137
137.5651233073087
153.23143575856207
158.49697553249007
185.54487583566825
237.87750268438478
154.9262697342237
227.2496082181753
213.30518318930316
194.38826893701534
154.51385094714874
175.45138362330218
128.3817935097073
165.44181434303036
151.0255979247089
218.30897260055008
123.3573041631081
160.72805473516433
125.48958974191297
167.4664496303456
133.6384696113068
122.84514876079956
99.85805216608479
143.47012268681053
136.96992191133418
141.8655837999177
129.32435010707638
107.62303232938191
126.5876131605322
90.25644731001152
153.45586149546918
126.57646854932386
100.39485311142657
97.10742825468513
86.34819476257108
77.39740118311974
104.86948598946456
109.14249412332187
124.86052873951428
206.8993578237229
275.1309401692711
128.6639016612282
80.70241280011066
92.43969055128163
87.28974039010158
95.39010872326446
95.04788171036797
71.74606262854033
83.95598815661984
83.6434782569339
87.25727647038599
100.044811552754

KeyboardInterrupt: 